In [1]:
import gradio as gr
import os
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

C:\Users\cscs8\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 載入模型


In [2]:
# 加載模型
model = tf.keras.models.load_model('optimized_pneumonia_model.keras')

# 定義分類函數
def classify_image(image):
    # 圖像處理
    image = np.array(image)
    if len(image.shape) == 2:
        image = np.stack((image,)*3, axis=-1)
    image = tf.image.resize(image, (150, 150)).numpy()
    image = image / 255.0
    image = np.expand_dims(image, axis=0)

    # 預測
    prediction = model.predict(image)
    prediction = prediction[0][0]

    # 返回結果
    if prediction > 0.5:
        return f"肺炎可能性: {round(prediction*100, 2)}% (檢測結果: 肺炎)"
    else:
        return f"肺炎可能性: {round(prediction*100, 2)}% (檢測結果: 正常)"


# 載入測試數據集中的圖像


In [3]:
def load_test_images(test_dir, label, num_images=10):
    directory = os.path.join(test_dir, label)
    images = os.listdir(directory)[:num_images]
    plt.figure(figsize=(15, 10))
    for i, image_name in enumerate(images, start=1):
        img = Image.open(os.path.join(directory, image_name))
        plt.subplot(2, 5, i)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    return plt


# 創建 Gradio 界面


In [4]:
def show_test_images(label):
    return load_test_images('chest_xray/test', label)

# 選擇每個類別的幾個示例圖像
normal_examples = [os.path.join('./chest_xray/test/NORMAL/', img) for img in os.listdir('./chest_xray/test/NORMAL/')[:5]]
pneumonia_examples = [os.path.join('./chest_xray/test/PNEUMONIA/', img) for img in os.listdir('./chest_xray/test/PNEUMONIA/')[:5]]

# Gradio 介面
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(shape=(150, 150)),
    outputs="text",
    title="肺炎檢測模型",
    description="上傳一張胸部X光圖像來檢測是否有肺炎。",
    examples=normal_examples + pneumonia_examples
)


# 啟動界面


In [5]:
iface.launch(share=False)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 18ms/step
